In [2]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt # for image plotting

# Any results you write to the current directory are saved as output.

# read input train.csv
train_set = pd.read_csv('train.csv').astype(np.float32)
test_set = pd.read_csv('test.csv').astype(np.float32)
print(train_set.shape)
print(test_set.shape)


(42000, 785)
(28000, 784)


In [3]:
#show i'th row of input as image.
#ignore first column as that is label
image_index = 41000
b = range(1,785)
image = np.reshape(train_set.values[image_index][b], (28,28))
print("labelled as ")
print(train_set.values[image_index][0])
plt.imshow(image, cmap = 'gray')
#image = np.reshape(train_set.values[0], (28,28))

#split training 90/10
mask = np.random.rand(len(train_set)) < 0.9
train_set_train = train_set[mask]
train_set_valid = train_set[~mask]

xs_valid = train_set_valid.values[:,b]
ys_valid = train_set_valid.values[:,0]

labelled as 
2.0


In [4]:
x = tf.placeholder(tf.float32, [None, 784])   #input placeholder

##### Model 2  - Deep MNIST #####
def weights(shape):
    var = tf.Variable(tf.truncated_normal(shape, stddev = 0.1))
    return var

def bias(shape):
    var = tf.constant(0.1, shape=shape)
    return tf.Variable(var)

def conv2d(x, W):
    conv = tf.nn.conv2d(x, W, strides = [1,1,1,1], padding = "SAME", name = "dummy")
    return conv

def max_pool2X2(x):
    maxpool = tf.nn.max_pool(x, ksize = [1,2,2,1], strides = [1,2,2,1], padding = "SAME")
    return maxpool

In [5]:
#first layer
weight_conv1 = weights([5,5,1,32])
bias_conv1 = bias([32])

train_data = tf.reshape(x, (-1,28,28,1))

conv1 = tf.nn.relu(conv2d(train_data, weight_conv1)+bias_conv1)
pool1 = max_pool2X2(conv1)

In [6]:
#second layer
weight_conv2 = weights([5,5,32,64])
bias_conv2 = bias([64])
conv2 = tf.nn.relu(conv2d(pool1, weight_conv2)+bias_conv2)
pool2 = max_pool2X2(conv2)

In [7]:
#fully connected
fc1weights = weights([7*7*64,1024])
fc1bias = weights([1024])

fcrelu = tf.nn.relu(tf.matmul(tf.reshape(pool2,[-1,7*7*64]),fc1weights)+fc1bias)

In [8]:
#drop out
keepprob = tf.placeholder(tf.float32)
dropout = tf.nn.dropout(fcrelu,keepprob)

In [9]:
#last layer with 10 outputs
weights_y = weights([1024,10])
bias_y = bias([10])
y_2 = tf.matmul(dropout,weights_y)+bias_y

In [10]:
##Training
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_2))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
#train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_2, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#interactive session
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
acc = 0
for cnt in range(20001):
    if cnt%100 == 0:
        print("in loop {} accuracy is {}".format(cnt,acc))
    #batch sampling of 500
    train_set_batch = train_set_train.sample(100)
    xs = train_set_batch.values[:,range(1,785)]
    ys = train_set_batch.values[:,0]
    ys = tf.one_hot(ys, depth = 10).eval()
    acc = sess.run([train_step,accuracy], feed_dict={x:xs, y_:ys, keepprob : 0.5})

in loop 0 accuracy is 0
in loop 100 accuracy is [None, 0.91000003]
in loop 200 accuracy is [None, 0.92000002]
in loop 300 accuracy is [None, 0.91000003]
in loop 400 accuracy is [None, 0.92000002]
in loop 500 accuracy is [None, 0.97000003]
in loop 600 accuracy is [None, 0.92000002]
in loop 700 accuracy is [None, 0.95999998]
in loop 800 accuracy is [None, 0.98000002]
in loop 900 accuracy is [None, 0.94999999]
in loop 1000 accuracy is [None, 0.98000002]
in loop 1100 accuracy is [None, 0.98000002]
in loop 1200 accuracy is [None, 0.99000001]
in loop 1300 accuracy is [None, 0.97000003]
in loop 1400 accuracy is [None, 0.94]
in loop 1500 accuracy is [None, 0.98000002]
in loop 1600 accuracy is [None, 0.94999999]
in loop 1700 accuracy is [None, 0.98000002]
in loop 1800 accuracy is [None, 0.97000003]
in loop 1900 accuracy is [None, 0.98000002]
in loop 2000 accuracy is [None, 0.97000003]
in loop 2100 accuracy is [None, 1.0]
in loop 2200 accuracy is [None, 0.94999999]
in loop 2300 accuracy is [None

in loop 19600 accuracy is [None, 0.98000002]
in loop 19700 accuracy is [None, 1.0]
in loop 19800 accuracy is [None, 1.0]
in loop 19900 accuracy is [None, 0.99000001]
in loop 20000 accuracy is [None, 1.0]


In [11]:
#evaluation
ys_valid = tf.one_hot(ys_valid, depth = 10).eval()
print(sess.run(accuracy, feed_dict={x:xs_valid, y_:ys_valid, keepprob :1.0}))

0.986089


In [21]:
#submission
print("starting prediction")
prediction = tf.argmax(y_2, 1)
predicted_values = sess.run(prediction, feed_dict={x:test_set[15000:len(test)], keepprob : 1.0})

my_submission = pd.DataFrame({ 'label': predicted_values})
# you could use any filename. We choose submission here
my_submission.to_csv('submission3.csv', index=False)

starting prediction
